In [1]:
import torch

In [8]:
!pip install torch==1.8.0 torchtext==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 1.8.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.8.0 which is incompatible.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 1.8.0 which is incompatible.


In [16]:
!pip install --upgrade torch

  Using cached torch-2.0.0-cp39-none-macosx_10_9_x86_64.whl (139.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 2.0.0 which is incompatible.


In [83]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(1)

# Define hyperparameters
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_LAYERS = 1
DROPOUT = 0.33
OUTPUT_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.1
EPOCHS = 50

# Define the model
class BLSTM(nn.Module):

    def __init__(self, vocab_size, tagset_size):
        super(BLSTM, self).__init__()
        self.embedding_dim = EMBEDDING_DIM
        self.hidden_dim = HIDDEN_DIM
        self.num_layers = NUM_LAYERS
        self.dropout = DROPOUT
        self.output_dim = OUTPUT_DIM
        
        self.word_embeddings = nn.Embedding(vocab_size, self.embedding_dim)
        self.blstm = nn.LSTM(self.embedding_dim, self.hidden_dim, 
                             num_layers=self.num_layers, bidirectional=True,
                             dropout=self.dropout)
        self.hidden2tag = nn.Linear(self.hidden_dim*2, self.output_dim)
        self.elu = nn.ELU()
        self.tag_projection = nn.Linear(self.output_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.blstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        elu_out = self.elu(tag_space)
        tag_scores = self.tag_projection(elu_out)
        return tag_scores
    
def train(model, optimizer, loss_function, train_data, dev_data):
    for epoch in range(EPOCHS):
        train_loss = 0.0
        num_correct = 0
        num_total = 0
        model.train()
        for sentence, tags in train_data:
            model.zero_grad()
            sentence = autograd.Variable(torch.LongTensor(sentence))
            targets = autograd.Variable(torch.LongTensor(tags))
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, targets)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            _, predicted_tags = torch.max(tag_scores, 1)
            num_correct += (predicted_tags == targets).sum().item()
            num_total += len(targets)

        precision, recall, f1 = evaluate(model, dev_data)
        print("Epoch {}: Train Loss: {:.5f}, Precision: {:.3f}, Recall: {:.3f}, F1 Score: {:.3f}" .format(epoch+1, train_loss, precision, recall, f1))

torch.save(model.state_dict(), 'blstm1.pt')


def evaluate(model, data):
    num_correct = 0
    num_total = 0
    correct = 0
    model.eval()
    with torch.no_grad():
        for sentence, tags in data:
            sentence = autograd.Variable(torch.LongTensor(sentence))
            targets = autograd.Variable(torch.LongTensor(tags))
            tag_scores = model(sentence)
            _, predicted_tags = torch.max(tag_scores, 1)
            num_correct += (predicted_tags == targets).sum().item()
            num_total += len(targets)
            correct += (predicted_tags == targets).sum().item() * 1.0 / len(targets)

    precision = correct / num_correct
    recall = correct / num_total
    f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1

# Read in the train and dev data
def read_data(file_path, has_tags=True):
    sentences = []
    tags = []
    with open(file_path) as f:
        sentence = []
        tags_sequence = []
        for line in f:
            if line == '\n':
                sentences.append(sentence)
                tags.append(tags_sequence)
                sentence = []
                tags_sequence = []
            else:
                line = line.strip().split()
                word = line[1]
                sentence.append(word)
                if has_tags:
                    tag = line[2]
                    tags_sequence.append(tag)
    return sentences, tags



# Define the paths to the train and dev data files
train_file_path = "train.txt"
dev_file_path = "dev.txt"
test_file_path = "test.txt"


# Read in the train and dev data
train_sentences, train_tags = read_data(train_file_path)
dev_sentences, dev_tags = read_data(dev_file_path)
test_sentences, _ = read_data(test_file_path, has_tags=False)

# Create the vocabulary and tag sets
word_to_ix = {}
tag_to_ix = {}
for sentence, tags in zip(train_sentences, train_tags):
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

# Add the unknown token to the vocabulary
word_to_ix["<UNK>"] = len(word_to_ix)

# Convert the sentences and tags to lists of indices
train_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), 
               torch.tensor([tag_to_ix[tag] for tag in tags])) 
              for sentence, tags in zip(train_sentences, train_tags)]
dev_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), 
             torch.tensor([tag_to_ix[tag] for tag in tags])) 
            for sentence, tags in zip(dev_sentences, dev_tags)]

test_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), None) 
            for sentence in test_sentences]
# Create the model
model = BLSTM(len(word_to_ix), len(tag_to_ix))

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# # Train the model
# train(model, optimizer, loss_function, train_data, dev_data)

# def predict(model, data, word_to_ix, tag_to_ix, file_path):
#     with open(file_path, "w") as f:
#         model.eval()
#         with torch.no_grad():
#             for i, (sentence, tags) in enumerate(data):
#                 sentence = autograd.Variable(sentence.view(-1, 1))
#                 tag_scores = model(sentence)
#                 _, predicted_tags = torch.max(tag_scores, 1)
#                 for j, word_index in enumerate(sentence):
#                     word = list(word_to_ix.keys())[list(word_to_ix.values()).index(word_index.item())]
#                     gold_tag = ''
#                     if tags is not None:
#                         gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
#                     pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
#                     f.write(str(j+1) + " " + word + " " + gold_tag + " " + pred_tag + "\n")
#                 f.write("\n")

def predict(model, original_sentences, data, word_to_ix, tag_to_ix, file_path, include_gold_tag=True):
    with open(file_path, "w") as f:
        model.eval()
        with torch.no_grad():
            for i, (sentence, tags) in enumerate(data):
                original_sentence = original_sentences[i]
                sentence = autograd.Variable(sentence.view(-1, 1))
                tag_scores = model(sentence)
                _, predicted_tags = torch.max(tag_scores, 1)
                for j, word in enumerate(original_sentence):
                    gold_tag = ''
                    if tags is not None and include_gold_tag:
                        gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
                    pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
                    f.write(str(j+1) + " " + word + " " + gold_tag + " " + pred_tag + "\n")
                f.write("\n")


if pred_file_path is not None:
    with open(pred_file_path, "a") as f_pred:
        for i, (sentence, tags) in enumerate(dev_data):
            sentence = autograd.Variable(sentence.view(-1, 1))
            tag_scores = model(sentence)
            _, predicted_tags = torch.max(tag_scores, 1)
            for j, word_index in enumerate(sentence):
                word = list(word_to_ix.keys())[list(word_to_ix.values()).index(word_index.item())]
                gold_tag = ''
                if tags is not None:
                    gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
                pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
                f_pred.write(str(j+1) + " " + word + " " + " " + pred_tag + "\n")
            f_pred.write("\n")
                
pred_file_path = "dev1.out"
perl_dev_file_path = "perl_dev1.out"
test_pred_file_path = "test1.out"

# Generate the predictions and write them to the prediction files
predict(model, dev_sentences, dev_data, word_to_ix, tag_to_ix, pred_file_path, include_gold_tag=False)
predict(model, dev_sentences, dev_data, word_to_ix, tag_to_ix, perl_dev_file_path, include_gold_tag=True)
predict(model, test_sentences, test_data, word_to_ix, tag_to_ix, test_pred_file_path, include_gold_tag=False)


Epoch 1: Train Loss: 6323.74657, Precision: 0.067, Recall: 0.060, F1 Score: 0.063
Epoch 2: Train Loss: 2971.23472, Precision: 0.067, Recall: 0.061, F1 Score: 0.064
Epoch 3: Train Loss: 1608.57225, Precision: 0.066, Recall: 0.062, F1 Score: 0.064
Epoch 4: Train Loss: 904.71808, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 5: Train Loss: 509.40945, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 6: Train Loss: 336.30222, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 7: Train Loss: 254.87348, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 8: Train Loss: 215.39131, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 9: Train Loss: 284.37480, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 10: Train Loss: 269.56684, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 11: Train Loss: 210.65988, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 12: Train Loss: 108.75902, Precision: 0.067, Recall: 0.063, F1 Score: 0.065
Epoch 13: Train Loss: 

# Task2

In [84]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import gzip
# Set random seed for reproducibility
torch.manual_seed(1)

# Define hyperparameters
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_LAYERS = 1
DROPOUT = 0.33
OUTPUT_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.1
EPOCHS = 10





def load_glove_embeddings(glove_path, word_to_ix, embed_dim):
    embeddings = {}
    with gzip.open(glove_path, 'rt', encoding='utf-8') as f:  # Use gzip.open instead of open
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = [float(val) for val in values[1:]]
            embeddings[word] = vector

    embedding_matrix = torch.zeros(len(word_to_ix), embed_dim)
    for word, index in word_to_ix.items():
        if word in embeddings:
            embedding_matrix[index] = torch.FloatTensor(embeddings[word])
        else:
            embedding_matrix[index] = torch.FloatTensor(embed_dim).uniform_(-0.5, 0.5)

    return embedding_matrix



# Define the model
class BLSTM(nn.Module):

    def __init__(self, vocab_size, tagset_size, pretrained_embeddings):
        super(BLSTM, self).__init__()
        self.embedding_dim = EMBEDDING_DIM
        self.hidden_dim = HIDDEN_DIM
        self.num_layers = NUM_LAYERS
        self.dropout = DROPOUT
        self.output_dim = OUTPUT_DIM
        
        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_embeddings)
        self.blstm = nn.LSTM(self.embedding_dim, self.hidden_dim, 
                             num_layers=self.num_layers, bidirectional=True,
                             dropout=self.dropout)
        self.hidden2tag = nn.Linear(self.hidden_dim*2, self.output_dim)
        self.elu = nn.ELU()
        self.tag_projection = nn.Linear(self.output_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.blstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        elu_out = self.elu(tag_space)
        tag_scores = self.tag_projection(elu_out)
        return tag_scores
    
# Define the training function
def train(model, optimizer, loss_function, train_data, dev_data):
    for epoch in range(EPOCHS):
        train_loss = 0.0
        num_correct = 0
        num_total = 0
        model.train()
        for sentence, tags in train_data:
            model.zero_grad()
            sentence = autograd.Variable(torch.LongTensor(sentence))
            targets = autograd.Variable(torch.LongTensor(tags))
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, targets)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            _, predicted_tags = torch.max(tag_scores, 1)
            num_correct += (predicted_tags == targets).sum().item()
            num_total += len(targets)

        precision, recall, f1 = evaluate(model, dev_data)
        print("Epoch {}: Train Loss: {:.5f}, Precision: {:.3f}, Recall: {:.3f}, F1 Score: {:.3f}" .format(epoch+1, train_loss, precision, recall, f1))

torch.save(model.state_dict(), 'blstm2.pt')

def evaluate(model, data):
    num_correct = 0
    num_total = 0
    correct = 0
    model.eval()
    with torch.no_grad():
        for sentence, tags in data:
            sentence = autograd.Variable(torch.LongTensor(sentence))
            targets = autograd.Variable(torch.LongTensor(tags))
            tag_scores = model(sentence)
            _, predicted_tags = torch.max(tag_scores, 1)
            num_correct += (predicted_tags == targets).sum().item()
            num_total += len(targets)
            correct += (predicted_tags == targets).sum().item() * 1.0 / len(targets)

    precision = correct / num_correct
    recall = correct / num_total
    f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1

# Read in the train and dev data
def read_data(file_path, has_tags=True):
    sentences = []
    tags = []
    with open(file_path) as f:
        sentence = []
        tags_sequence = []
        for line in f:
            if line == '\n':
                sentences.append(sentence)
                tags.append(tags_sequence)
                sentence = []
                tags_sequence = []
            else:
                line = line.strip().split()
                word = line[1]
                sentence.append(word)
                if has_tags:
                    tag = line[2]
                    tags_sequence.append(tag)
    return sentences, tags


glove_path = "glove.6B.100d.gz"
pretrained_embeddings = load_glove_embeddings(glove_path, word_to_ix, EMBEDDING_DIM)

# Define the paths to the train and dev data files
train_file_path = "train.txt"
dev_file_path = "dev.txt"
test_file_path = "test.txt"


# Read in the train and dev data
train_sentences, train_tags = read_data(train_file_path)
dev_sentences, dev_tags = read_data(dev_file_path)
test_sentences, _ = read_data(test_file_path, has_tags=False)

# Create the vocabulary and tag sets
word_to_ix = {}
tag_to_ix = {}
for sentence, tags in zip(train_sentences, train_tags):
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

# Add the unknown token to the vocabulary
word_to_ix["<UNK>"] = len(word_to_ix)

# Convert the sentences and tags to lists of indices
train_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), 
               torch.tensor([tag_to_ix[tag] for tag in tags])) 
              for sentence, tags in zip(train_sentences, train_tags)]
dev_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), 
             torch.tensor([tag_to_ix[tag] for tag in tags])) 
            for sentence, tags in zip(dev_sentences, dev_tags)]

test_data = [(torch.tensor([word_to_ix.get(word, word_to_ix["<UNK>"]) for word in sentence]), None) 
            for sentence in test_sentences]
# Create the model
model = BLSTM(len(word_to_ix), len(tag_to_ix), pretrained_embeddings)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# Train the model
train(model, optimizer, loss_function, train_data, dev_data)

# def predict(model, data, word_to_ix, tag_to_ix, file_path):
#     with open(file_path, "w") as f:
#         model.eval()
#         with torch.no_grad():
#             for i, (sentence, tags) in enumerate(data):
#                 sentence = autograd.Variable(sentence.view(-1, 1))
#                 tag_scores = model(sentence)
#                 _, predicted_tags = torch.max(tag_scores, 1)
#                 for j, word_index in enumerate(sentence):
#                     word = list(word_to_ix.keys())[list(word_to_ix.values()).index(word_index.item())]
#                     gold_tag = ''
#                     if tags is not None:
#                         gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
#                     pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
#                     f.write(str(j+1) + " " + word + " " + gold_tag + " " + pred_tag + "\n")
#                 f.write("\n")

def predict(model, original_sentences, data, word_to_ix, tag_to_ix, file_path, include_gold_tag=True):
    with open(file_path, "w") as f:
        model.eval()
        with torch.no_grad():
            for i, (sentence, tags) in enumerate(data):
                original_sentence = original_sentences[i]
                sentence = autograd.Variable(sentence.view(-1, 1))
                tag_scores = model(sentence)
                _, predicted_tags = torch.max(tag_scores, 1)
                for j, word in enumerate(original_sentence):
                    gold_tag = ''
                    if tags is not None and include_gold_tag:
                        gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
                    pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
                    f.write(str(j+1) + " " + word + " " + gold_tag + " " + pred_tag + "\n")
                f.write("\n")


if pred_file_path is not None:
    with open(pred_file_path, "a") as f_pred:
        for i, (sentence, tags) in enumerate(dev_data):
            sentence = autograd.Variable(sentence.view(-1, 1))
            tag_scores = model(sentence)
            _, predicted_tags = torch.max(tag_scores, 1)
            for j, word_index in enumerate(sentence):
                word = list(word_to_ix.keys())[list(word_to_ix.values()).index(word_index.item())]
                gold_tag = ''
                if tags is not None:
                    gold_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tags[j].item())]
                pred_tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(predicted_tags[j].item())]
                f_pred.write(str(j+1) + " " + word + " " + " " + pred_tag + "\n")
            f_pred.write("\n")
                
pred_file_path = "dev2.out"
perl_dev_file_path = "perl_dev2.out"
test_pred_file_path = "test2.out"

# Generate the predictions and write them to the prediction files
predict(model, dev_sentences, dev_data, word_to_ix, tag_to_ix, pred_file_path, include_gold_tag=False)
predict(model, dev_sentences, dev_data, word_to_ix, tag_to_ix, perl_dev_file_path, include_gold_tag=True)
predict(model, test_sentences, test_data, word_to_ix, tag_to_ix, test_pred_file_path, include_gold_tag=False)



Epoch 1: Train Loss: 4962.93867, Precision: 0.066, Recall: 0.058, F1 Score: 0.061
Epoch 2: Train Loss: 3057.27422, Precision: 0.066, Recall: 0.059, F1 Score: 0.062
Epoch 3: Train Loss: 2504.55590, Precision: 0.066, Recall: 0.060, F1 Score: 0.063
Epoch 4: Train Loss: 2121.24326, Precision: 0.066, Recall: 0.060, F1 Score: 0.063
Epoch 5: Train Loss: 1805.53576, Precision: 0.066, Recall: 0.060, F1 Score: 0.063
Epoch 6: Train Loss: 1536.03511, Precision: 0.066, Recall: 0.060, F1 Score: 0.063
Epoch 7: Train Loss: 1307.10273, Precision: 0.066, Recall: 0.061, F1 Score: 0.063
Epoch 8: Train Loss: 1107.94243, Precision: 0.066, Recall: 0.061, F1 Score: 0.063
Epoch 9: Train Loss: 940.60302, Precision: 0.067, Recall: 0.061, F1 Score: 0.064
Epoch 10: Train Loss: 790.19373, Precision: 0.067, Recall: 0.061, F1 Score: 0.064
Epoch 11: Train Loss: 684.41277, Precision: 0.067, Recall: 0.061, F1 Score: 0.064
Epoch 12: Train Loss: 568.95595, Precision: 0.067, Recall: 0.061, F1 Score: 0.064
Epoch 13: Train L

In [86]:
import torch
from torch.autograd import Variable

# Create a new instance of the model with the same architecture
loaded_model = BLSTM(len(word_to_ix), len(tag_to_ix), pretrained_embeddings)

# Load the saved state dictionary into the new model
loaded_model.load_state_dict(torch.load('blstm2.pt'))

# Set the model to evaluation mode
loaded_model.eval()

BLSTM(
  (word_embeddings): Embedding(23625, 100)
  (blstm): LSTM(100, 256, dropout=0.33, bidirectional=True)
  (hidden2tag): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (tag_projection): Linear(in_features=128, out_features=9, bias=True)
)

In [87]:
dev_pred_file_path = "dev_predictions.txt"
test_pred_file_path = "test_predictions.txt"

# Generate the predictions and write them to the prediction files
predict(loaded_model, dev_sentences, dev_data, word_to_ix, tag_to_ix, dev_pred_file_path)
predict(loaded_model, test_sentences, test_data, word_to_ix, tag_to_ix, test_pred_file_path)

In [88]:
import torch
from torch.autograd import Variable

# Create a new instance of the model with the same architecture
loaded_model = BLSTM(len(word_to_ix), len(tag_to_ix))

# Load the saved state dictionary into the new model
loaded_model.load_state_dict(torch.load('blstm1.pt'))

# Set the model to evaluation mode
loaded_model.eval()



torch.save(model_lstm.state_dict(), 'blstm1.pt')
loaded_model_lstm = LSTM(vocab_size = len(vocab)+1, embedding_dim = 100, lstm_hidden_size = 256, fc_num_neurons = 128, num_classes = len(labels)).to(device)
loaded_model_lstm.load_state_dict(torch.load('blstm1.pt'))

BLSTM(
  (word_embeddings): Embedding(23625, 100)
  (blstm): LSTM(100, 256, dropout=0.33, bidirectional=True)
  (hidden2tag): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (tag_projection): Linear(in_features=128, out_features=9, bias=True)
)

In [89]:
dev_pred_file_path = "dev_predictions1.txt"
test_pred_file_path = "test_predictions1.txt"

# Generate the predictions and write them to the prediction files
predict(loaded_model, dev_sentences, dev_data, word_to_ix, tag_to_ix, dev_pred_file_path)
predict(loaded_model, test_sentences, test_data, word_to_ix, tag_to_ix, test_pred_file_path)

In [91]:
print(len(word_to_ix))

23625
